# Dataset Factory
Notebook for using the xrfuncs module to combine spice-2 simlulation results into single datasets for analysis. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 14
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu
import flopter.spice.xrfuncs as xrf

# Tool for globbing together the run directories you want to combine

This 

In [2]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [3]:
skippable_runs = set([
    'marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-6.0', # accidental duplicate
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-4.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-5.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-6.0',
    'marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-2.0',
    'marconi/spice2/bergmann_bm/flat_flush_lowas/alpha_yz_-1.5',
#     'marconi/spice2/shexp_shad_min/flat_flush_as/alpha_yz_-4.0', # unfinished
    'marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-4.0', 
    'marconi/spice2/shexp_shad_fwp0/flat_flush_as/alpha_yz_-4.0'
]) 
skippable_scans = set()
single_sims = set()

In [4]:
if 1 == 0:
    sr_sorted = list(skippable_runs)
    sr_sorted.sort()
    for skippable_run in sr_sorted:
        backup_dir = list(pth.Path(skippable_run).glob('backup*'))[-1]
    #     print(backup_dir/'log.out')
        print(f'{skippable_run}:')
        print(f'\t{spl.Splopter.get_h_remaining_lines(backup_dir/"log.out")[-1]}\n')

In [5]:
non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

In [6]:
# scans_searchstr = '*/*/sheath_exp/*'
# scans_searchstr = '*/*/sheath_exp_fwp/*'
scans_searchstr = [
#     '*/*/sheath_exp_hg/angled_recessed_as', 
#     '*/*/sheath_exp_hg/flat_flush*',
#     '*/*/sheath_exp_hg/*',
#     '*/*/sheath_exp_hg_fflwp/*'
#     '*/*/sheath_exp_fflwp/*'
#     '*/*/sheath_exp_fwp/*wp-2*',
#     '*/*/sheath_exp_fwp/flat_flush_as'
#     '*/*/bergmann_bm/*'
    
    '*/*/shexp_shad_fflwp*/*',
    '*/*/shexp_shad_min/*',
#     '*/*/shexp_shad_fwp0/*',

]
# disallowed_angles = ['-2.0', '-3.0', '-4.0', '-5.0', '-6.0']
disallowed_angles = ['-2.0', '-3.0']

scans, all_run_dirs = xrf.get_run_dirs(scans_searchstr, skippable_runs=skippable_runs, 
                                       disallowed_angles=disallowed_angles)

[0]: marconi/spice2/shexp_shad_fflwp/angled_recessed_as
	[0,0]: angled_recessed_as/alpha_yz_-11.0
	[0,1]: angled_recessed_as/alpha_yz_-12.0
	[0,2]: angled_recessed_as/alpha_yz_-14.0
	[0,3]: angled_recessed_as/alpha_yz_-16.0
	[0,4]: angled_recessed_as/alpha_yz_-18.0
	[0,5]: angled_recessed_as/alpha_yz_-20.0
	[0,6]: angled_recessed_as/alpha_yz_-25.0
	[0,7]: angled_recessed_as/alpha_yz_-30.0
	[0,8]: angled_recessed_as/alpha_yz_-5.0
	[0,9]: angled_recessed_as/alpha_yz_-7.0
	[0,10]: angled_recessed_as/alpha_yz_-8.0
	[0,11]: angled_recessed_as/alpha_yz_-9.0
[1]: marconi/spice2/shexp_shad_fflwp/flat_flush_as
	[1,0]: flat_flush_as/alpha_yz_-11.0
	[1,1]: flat_flush_as/alpha_yz_-12.0
	[1,2]: flat_flush_as/alpha_yz_-14.0
	[1,3]: flat_flush_as/alpha_yz_-16.0
	[1,4]: flat_flush_as/alpha_yz_-18.0
	[1,5]: flat_flush_as/alpha_yz_-20.0
	[1,6]: flat_flush_as/alpha_yz_-25.0
	[1,7]: flat_flush_as/alpha_yz_-30.0
	[1,8]: flat_flush_as/alpha_yz_-5.0
	[1,9]: flat_flush_as/alpha_yz_-7.0
	[1,10]: flat_flush_as/

## The function itself

In [7]:
importlib.reload(xrf)

<module 'flopter.spice.xrfuncs' from '/home/jleland/coding/projects/flopter/flopter/spice/xrfuncs.py'>

In [8]:
datasets, probes, thetas = xrf.create_scan_probe_datasets(scans, all_run_dirs)



 --- marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-11.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-11.0/backup_20210322-0104) doesn't seem to be valid.
Continuing anyway.
No a-file given, continuing without


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)




 --- marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-12.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-12.0/backup_20210322-0235) doesn't seem to be valid.
Continuing anyway.
No a-file given, continuing without


 --- marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-14.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-14.0/backup_20210321-1512) doesn't seem to be valid.
Continuing anyway.
No a-file given, continuing without


 --- marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-16.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-1

/home/jleland/coding/projects/flopter/flopter/core/fitters.py:248: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))


angled_recessed 12.0
angled_recessed 14.0
angled_recessed 16.0


/home/jleland/coding/projects/flopter/flopter/core/fitters.py:248: RuntimeWarning: overflow encountered in multiply
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))


angled_recessed 18.0
angled_recessed 20.0
angled_recessed 25.0
angled_recessed 30.0
angled_recessed 5.0
angled_recessed 7.0
angled_recessed 8.0


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:461: RuntimeWarning: overflow encountered in multiply
  return transform * (func(xdata, *params) - ydata)


angled_recessed 9.0
flat_flush 11.0
flat_flush 12.0
flat_flush 14.0
flat_flush 16.0
flat_flush 18.0
flat_flush 20.0
flat_flush 25.0
flat_flush 30.0
flat_flush 5.0
flat_flush 7.0
flat_flush 8.0
flat_flush 9.0
angled_recessed 10.0
angled_recessed 4.0
angled_recessed 6.0
flat_flush 10.0
flat_flush 4.0
flat_flush 6.0


In [23]:
datasets.keys()

dict_keys(['angled_recessed', 'flat_flush'])

## Combining together the individual datasets
This has been implemented as a do-all function, done by combining all groups (i.e. folders in bin/data/) as datasets 2D in probe name (i.e. angled_recessed_...) and theta. 
These can then be further combined if desired. 

In [10]:
## DO NOT USE! These have now been implemented in xrfuncs and so are obsolete.

probe_theta_ps = {
    'angled':10.0,
    'flat':0.0,
    'semi-angled':5.0,
}
probe_recessions = {
    'recessed': 1.0e-3,
    'semi-recessed': 0.5e-3,
    'flush': 0.0e-3,
}

def combine_1d_dataset(probe_name, datasets, concat_dim='theta', theta_p='auto', recession='auto'):
    combined_ds = xr.concat(datasets[probe_name], dim=concat_dim).sortby(concat_dim)
    
    if theta_p == 'auto':
        theta_p = probe_theta_ps[probe_name.split('_')[0]]
    
    if recession == 'auto':
        recession = probe_recessions[probe_name.split('_')[1]]
    
    gap = 0.0 if 'gapless' in probe_name else 1.0e-3
    
    combined_ds = combined_ds.assign_coords(
        recession=recession,
        gap=gap,
        theta_p=theta_p,
        theta_p_rads=np.radians(theta_p),
        theta_rads=np.radians(combined_ds.theta)
    )
    return combined_ds

def combine_2d_dataset(probe_names, datasets, concat_dim='probe', ):
    c1d_datasets = [combine_1d_dataset(probe_name, datasets) for probe_name in probe_names]
    probe_da = xr.DataArray(probe_names, dims='probe', coords={'probe': probe_names})
    return xr.concat(c1d_datasets, dim=probe_da).drop(None)

In [9]:
probes

['angled_recessed', 'flat_flush']

In [10]:
combined_ds = xrf.combine_1d_dataset('flat_flush', datasets)
combineder_ds = xrf.combine_2d_dataset(probes, datasets, extra_dims={'run':'hg'})
# combineder_ds.sel(probe='angled_recessed')

In [11]:
combined_ds

<xarray.Dataset>
Dimensions:               (theta: 15, voltage: 801)
Coordinates:
  * voltage               (voltage) float64 -14.97 -14.95 -14.92 ... 5.0 5.025
  * theta                 (theta) float64 4.0 5.0 6.0 7.0 ... 20.0 25.0 30.0
    v_f                   (theta) float64 -4.452 -4.446 -4.354 ... -2.268 -1.685
    recession             float64 0.0
    gap                   float64 0.001
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.06981 0.08727 ... 0.4363 0.5236
    recession_descr       <U5 'flush'
    gap_desc              <U1 ''
    theta_p_descr         <U4 'flat'
    recessed_fl           bool False
    angled_fl             bool False
Data variables:
    str_iv_chi2           (theta) float64 661.0 554.8 636.3 ... 679.9 628.4
    str_iv_reduced_chi2   (theta) float64 1.585 1.33 1.515 ... 1.26 1.349 1.192
    str_iv_I_sat          (theta) float64 -12.92 -14.01 -17.05 ... -70.07 -89.42
    str_iv_d_I_sat        (theta) float64 0.1225 0.1121 0.1239 ... 0.3406 0.4116
    str_iv_a              (theta) float64 0.06511 0.06014 ... 0.009394 0.008253
    str_iv_d_a            (theta) float64 0.001801 0.001444 ... 0.0005545
    str_iv_T_e            (theta) float64 0.7626 0.6173 0.507 ... 0.4303 0.5167
    str_iv_d_T_e          (theta) float64 0.02268 0.0184 ... 0.01421 0.01452
    str_iv_V_f            (theta) float64 -4.513 -4.478 -4.364 ... -2.236 -1.644
    str_iv_d_V_f          (theta) float64 0.0106 0.009668 ... 0.01043 0.01071
    str_iv_run_dir        (theta) <U60 'marconi/spice2/shexp_shad_min/flat_flush_as/alpha_yz_-4.0' ... 'marconi/spice2/shexp_shad_fflwp/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (theta) float64 -14.97 -14.97 -14.97 ... -14.97 -14.97
    str_iv_voltage_max    (theta) float64 -4.475 -4.45 -4.375 ... -2.275 -1.7
    norm_iv_chi2          (theta) float64 808.0 1.121e+03 ... 3.484e+03
    norm_iv_reduced_chi2  (theta) float64 1.928 2.675 5.025 ... 8.816 6.585
    norm_iv_I_sat         (theta) float64 -13.91 -16.13 -20.36 ... -77.74 -97.27
    norm_iv_d_I_sat       (theta) float64 0.0944 0.1288 0.2059 ... 0.8483 0.8804
    norm_iv_a             (theta) float64 0.05838 0.04624 ... 4.104e-15
    norm_iv_d_a           (theta) float64 0.002111 0.0024 ... 0.002405 0.001934
    norm_iv_voltage_min   (theta) float64 -10.52 -10.53 -10.62 ... -12.71 -13.29
    norm_iv_voltage_max   (theta) float64 -0.0234 -0.004412 ... -0.01484
    ion_chi2              (theta) float64 765.3 545.4 671.5 ... 632.5 616.5
    ion_reduced_chi2      (theta) float64 1.844 1.314 1.606 ... 1.169 1.26 1.174
    ion_I_sat             (theta) float64 12.35 13.78 16.76 ... 53.79 71.16 90.0
    ion_d_I_sat           (theta) float64 0.05983 0.05881 ... 0.2318 0.2758
    ion_a                 (theta) float64 0.09339 0.09126 ... 0.01444 0.01226
    ion_d_a               (theta) float64 0.001774 0.00156 ... 0.0007292
    ion_voltage_min       (theta) float64 0.2082 0.1837 0.2057 ... 0.1869 0.1973
    ion_voltage_max       (theta) float64 5.843 5.845 5.884 ... 6.548 6.73 6.96
    elec_chi2             (theta) float64 1.87e+03 435.7 ... 1.964e+03 691.4
    elec_reduced_chi2     (theta) float64 4.464 1.04 0.4792 ... 3.55 3.881 1.307
    elec_T_e              (theta) float64 0.7988 0.5979 0.5186 ... 0.4652 0.5283
    elec_d_T_e            (theta) float64 0.007437 0.002904 ... 0.003122
    elec_I_sat            (theta) float64 -3.172e+03 -2.467e+04 ... -2.029e+03
    elec_d_I_sat          (theta) float64 182.1 960.6 2.525e+03 ... 526.7 45.91
    elec_voltage_min      (theta) float64 -14.97 -14.97 -14.97 ... -14.97 -14.97
    elec_voltage_max      (theta) float64 -4.475 -4.45 -4.375 ... -2.275 -1.7
    mf_iv_chi2            (theta) float64 754.7 623.5 556.5 ... 763.6 708.5
    mf_iv_reduced_chi2    (theta) float64 1.696 1.443 1.608 ... 1.471 1.305
    mf_iv_I_sat           (theta) float64 -13.26 -14.22 -18.11 ... -70.46 -88.83
    mf_iv_d_I_sat         (theta) f

In [27]:
fig, ax = plt.subplots(3, sharex=True, figsize=[8,8])
# fig = plt.figure(figsize=[8,8])
plot_ds = combineder_ds.sel(probe='flat_flush', voltage=slice(-15,None)).set_coords('voltage_corr')
#.swap_dims({'voltage':'voltage_corr'}) 
#, theta=[4.0, 6.0, 8.0, 12.0])
plot_ds.current.plot(hue='theta', x='voltage_corr', ax=ax[0])

plot_ds.current_e.plot(hue='theta', x='voltage_corr', ax=ax[1])
plot_ds.current_i.plot(hue='theta', x='voltage_corr', ax=ax[2])
# .current.plot.line(hue='theta', x='voltage', ax=ax), col=['current', 'current_e', 'current_i']

for axis in ax:
    axis.get_legend().remove()

fig.tight_layout()

In [28]:
combined_ds.v_f.plot.line(x='theta')

In [29]:
combined_ds.ion_I_sat.plot.line(x='theta')

In [30]:
fig, ax = plt.subplots(2)
dummy_theta = np.linspace(2, 45.0, 5000)


for i, probe in enumerate(combineder_ds.probe.values):
    plot_ds = combineder_ds.sel(probe=probe, run='hg')
    ax[i].errorbar(plot_ds['theta_p']+plot_ds['theta'], plot_ds['ion_a'], yerr=plot_ds['ion_d_a'], fmt='.')
    
    calced_a = lpu.calc_new_sheath_expansion_param(
        5.0, 1e18, 5e-3, 1e-3, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values, 
#         c_1=0.5, c_2=0.6,
        c_1=0.9, c_2=0.6,
    #     c_1=1.4, c_2=0.39, # from hg-theta=15-30
#         c_1=2.0, c_2=0.14, # from hg-theta=11-30
    )
    ax[i].errorbar(dummy_theta, calced_a, label=r'Predicted - $\theta_{large}$', fmt='-', 
                linewidth=0.8, alpha=0.6)
    ax[i].set_ylim(0,0.15)

In [17]:
combined_ds['theta_p'] = 10.0
combined_ds = combined_ds.assign_coords(
    theta_p_rads=np.radians(combined_ds.theta_p),
    theta_rads=np.radians(combined_ds.theta)
)

In [ ]:
combined_ds.to_netcdf('sheath_exp_hg_ar_ivs.nc')


## Combine several groups together through

In [12]:
# group_name_searchstrings = {
#     'hg': ['*/*/sheath_exp_hg/*'],
#     'hg_fflwp': ['*/*/sheath_exp_hg_fflwp/*'],
#     'fwp_2': ['*/*/sheath_exp_fwp/*wp-2*'],
#     'fwp_0': ['*/*/sheath_exp_fwp/*_as'],
#     'fflwp': ['*/*/sheath_exp_fflwp/*'],
# #     'old': ['*/*/sheath_exp'],
# #     'new': ['*/*/new_sheath_exp'],
# #     'bbm': ['*/*/bergmann_bm/*'],
# }

group_name_searchstrings = {
    'fwp_0': ['*/*/shexp_shad_fwp0/*'],
    'fflwp': ['*/*/shexp_shad_fflwp/*', '*/*/shexp_shad_min/*'],
#     'old': '*/*/sheath_exp',
#     'new': '*/*/new_sheath_exp',
#     'bbm': '*/*/bergmann_bm/*',
}

In [13]:
for group, searchstr in group_name_searchstrings.items():
    print(f'{group}:{searchstr}')
    scans, all_run_dirs = xrf.get_run_dirs(searchstr, skippable_runs=skippable_runs, 
                                           disallowed_angles=disallowed_angles)

fwp_0:['*/*/shexp_shad_fwp0/*']
[0]: marconi/spice2/shexp_shad_fwp0/angled_recessed_as
	[0,0]: angled_recessed_as/alpha_yz_-10.0
	[0,1]: angled_recessed_as/alpha_yz_-11.0
	[0,2]: angled_recessed_as/alpha_yz_-12.0
	[0,3]: angled_recessed_as/alpha_yz_-14.0
	[0,4]: angled_recessed_as/alpha_yz_-16.0
	[0,5]: angled_recessed_as/alpha_yz_-18.0
	[0,6]: angled_recessed_as/alpha_yz_-20.0
	[0,7]: angled_recessed_as/alpha_yz_-25.0
	[0,8]: angled_recessed_as/alpha_yz_-30.0
	[0,9]: angled_recessed_as/alpha_yz_-5.0
	[0,10]: angled_recessed_as/alpha_yz_-6.0
	[0,11]: angled_recessed_as/alpha_yz_-7.0
	[0,12]: angled_recessed_as/alpha_yz_-8.0
	[0,13]: angled_recessed_as/alpha_yz_-9.0
[1]: marconi/spice2/shexp_shad_fwp0/flat_flush_as
	[1,0]: flat_flush_as/alpha_yz_-10.0
	[1,1]: flat_flush_as/alpha_yz_-11.0
	[1,2]: flat_flush_as/alpha_yz_-12.0
	[1,3]: flat_flush_as/alpha_yz_-14.0
	[1,4]: flat_flush_as/alpha_yz_-16.0
	[1,5]: flat_flush_as/alpha_yz_-18.0
	[1,6]: flat_flush_as/alpha_yz_-20.0
	[1,7]: flat_flus

In [14]:
datasets = []
for group, searchstr in group_name_searchstrings.items():
    print(f'{group}:{searchstr}')
    scans, all_run_dirs = xrf.get_run_dirs(searchstr, 
                                           skippable_runs=skippable_runs, 
                                           disallowed_angles=disallowed_angles, 
                                           print_fl=False)
    combined_ds = xrf.create_scan_dataset(scans, all_run_dirs, extra_dims={'run':group})
    datasets.append(combined_ds)


fwp_0:['*/*/shexp_shad_fwp0/*']


 --- marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-10.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-10.0/backup_20210323-1115) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


No a-file given, continuing without


 --- marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-11.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-11.0/backup_20210323-0920) doesn't seem to be valid.
Continuing anyway.
No a-file given, continuing without


 --- marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-12.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-12.0/backup_20210323-0800) doesn't seem to be valid.
Continuing anyway.
No a-file given, continuing without


 --- marconi/spice2/shexp_shad_fwp0/angled_recessed_as/alpha_yz_-14.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fwp0/

/home/jleland/coding/projects/flopter/flopter/core/fitters.py:248: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))
/home/jleland/coding/projects/flopter/flopter/core/fitters.py:248: RuntimeWarning: overflow encountered in multiply
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))


flat_flush 20.0
flat_flush 25.0
flat_flush 30.0
flat_flush 5.0
flat_flush 6.0
flat_flush 7.0
flat_flush 8.0
flat_flush 9.0
fflwp:['*/*/shexp_shad_fflwp/*', '*/*/shexp_shad_min/*']


 --- marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-11.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-11.0/backup_20210322-0104) doesn't seem to be valid.
Continuing anyway.


...

angled_recessed 12.0
angled_recessed 14.0
angled_recessed 16.0


/home/jleland/coding/projects/flopter/flopter/core/fitters.py:248: RuntimeWarning: overflow encountered in multiply
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))


angled_recessed 18.0
angled_recessed 20.0
angled_recessed 25.0
angled_recessed 30.0
angled_recessed 5.0
angled_recessed 7.0
angled_recessed 8.0


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:461: RuntimeWarning: overflow encountered in multiply
  return transform * (func(xdata, *params) - ydata)


angled_recessed 9.0
flat_flush 11.0
flat_flush 12.0
flat_flush 14.0
flat_flush 16.0
flat_flush 18.0
flat_flush 20.0
flat_flush 25.0
flat_flush 30.0
flat_flush 5.0
flat_flush 7.0
flat_flush 8.0
flat_flush 9.0
angled_recessed 10.0
angled_recessed 4.0
angled_recessed 6.0
flat_flush 10.0
flat_flush 4.0
flat_flush 6.0


In [15]:
# datasets_dir = pth.Path('/home/jleland/data/external_big/spice/sheath_exp_datasets')
# datasets_dir = pth.Path('/home/jleland/data/external_big/spice/sheath_exp_datasets/10V_cap')
datasets_dir = pth.Path('/home/jleland/data/external_big/spice/shexp_datasets')
os.chdir(datasets_dir)

In [16]:
# A couple of lines to add 'probe' to the datasets that were missing them (as they were 1d)
# This will no longer be necessary

# datasets[2] = datasets[2].expand_dims(dim=['probe']).assign_coords(probe=['flat_flush'])
# datasets[2] 
# datasets[5] = datasets[5].expand_dims(dim=['probe']).assign_coords(probe=['flat_flush_bbm'])
# datasets[5]

In [17]:
for ds in datasets:
    run = ds.run.values[0]
    print(run)
    ds.to_netcdf(f'se_{run}_ivs.nc')

fwp_0
fflwp


In [18]:
for i, ds in enumerate(datasets):
    run = ds.run.values[0]
    print(f'[{i}]: {run}')
    print(ds.dims)

[0]: fwp_0
Frozen(SortedKeysDict({'probe': 2, 'theta': 14, 'voltage': 801, 'run': 1}))
[1]: fflwp
Frozen(SortedKeysDict({'probe': 2, 'theta': 15, 'voltage': 801, 'run': 1}))


In [19]:
combined_ds = xr.concat(datasets, dim='run')
combined_ds

<xarray.Dataset>
Dimensions:               (probe: 2, run: 2, theta: 15, voltage: 801)
Coordinates:
    gap                   float64 0.001
    theta_p_rads          (probe) float64 0.1745 0.0
    recession_descr       (probe) <U8 'recessed' 'flush'
    theta_p               (probe) float64 10.0 0.0
    gap_desc              <U1 ''
    recessed_fl           (probe) bool True False
    recession             (probe) float64 0.0003 0.0
    theta_p_descr         (probe) <U6 'angled' 'flat'
    angled_fl             (probe) bool True False
  * voltage               (voltage) float64 -14.97 -14.95 -14.92 ... 5.0 5.025
  * theta                 (theta) float64 4.0 5.0 6.0 7.0 ... 20.0 25.0 30.0
  * probe                 (probe) object 'angled_recessed' 'flat_flush'
    theta_rads            (run, theta) float64 nan 0.08727 ... 0.4363 0.5236
    v_f                   (run, probe, theta) float64 nan -10.7 ... -1.685
  * run                   (run) object 'fwp_0' 'fflwp'
Data variables:
    str_iv_chi2           (run, probe, theta) float64 nan 155.1 ... 679.9 628.4
    str_iv_reduced_chi2   (run, probe, theta) float64 nan 0.9235 ... 1.349 1.192
    str_iv_I_sat          (run, probe, theta) float64 nan -38.02 ... -89.42
    str_iv_d_I_sat        (run, probe, theta) float64 nan 48.69 ... 0.4116
    str_iv_a              (run, probe, theta) float64 nan 2.061e-13 ... 0.008253
    str_iv_d_a            (run, probe, theta) float64 nan 0.3791 ... 0.0005545
    str_iv_T_e            (run, probe, theta) float64 nan 3.305 ... 0.5167
    str_iv_d_T_e          (run, probe, theta) float64 nan 12.83 ... 0.01452
    str_iv_V_f            (run, probe, theta) float64 nan -2.877 ... -1.644
    str_iv_d_V_f          (run, probe, theta) float64 nan 15.15 ... 0.01071
    str_iv_run_dir        (run, probe, theta) object nan ... 'marconi/spice2/shexp_shad_fflwp/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (run, probe, theta) float64 nan -14.97 ... -14.97
    str_iv_voltage_max    (run, probe, theta) float64 nan -10.7 ... -2.275 -1.7
    norm_iv_chi2          (run, probe, theta) float64 nan ... 3.484e+03
    norm_iv_reduced_chi2  (run, probe, theta) float64 nan 255.5 ... 8.816 6.585
    norm_iv_I_sat         (run, probe, theta) float64 nan -42.7 ... -97.27
    norm_iv_d_I_sat       (run, probe, theta) float64 nan 5.264 ... 0.8804
    norm_iv_a             (run, probe, theta) float64 nan 1.58e-21 ... 4.104e-15
    norm_iv_d_a           (run, probe, theta) float64 nan 0.05339 ... 0.001934
    norm_iv_voltage_min   (run, probe, theta) float64 nan -4.275 ... -13.29
    norm_iv_voltage_max   (run, probe, theta) float64 nan -0.025 ... -0.01484
    ion_chi2              (run, probe, theta) float64 nan 145.9 ... 632.5 616.5
    ion_reduced_chi2      (run, probe, theta) float64 nan 0.8842 ... 1.26 1.174
    ion_I_sat             (run, probe, theta) float64 nan 34.52 ... 71.16 90.0
    ion_d_I_sat           (run, probe, theta) float64 nan 0.1183 ... 0.2758
    ion_a                 (run, probe, theta) float64 nan 0.02572 ... 0.01226
    ion_d_a               (run, probe, theta) float64 nan 0.001897 ... 0.0007292
    ion_voltage_min       (run, probe, theta) float64 nan 0.2102 ... 0.1973
    ion_voltage_max       (run, probe, theta) float64 nan 2.973 ... 6.73 6.96
    elec_chi2             (run, probe, theta) float64 nan 2.195e-28 ... 691.4
    elec_reduced_chi2     (run, probe, theta) float64 nan 1.299e-30 ... 1.307
    elec_T_e              (run, probe, theta) float64 nan 4.059 ... 0.5283
    elec_d_T_e            (run, probe, theta) float64 nan 9.206e-31 ... 0.003122
    elec_I_sat            (run, probe, theta) float64 nan ... -2.029e+03
    elec_d_I_sat          (run, probe, theta) float64 nan 3.101e-16 ... 45.91
    elec_voltage_min      (run, probe, theta) float64 nan -14.97 ... -14.97
    elec_voltage_max      (run, probe, theta) float64 nan -10.72 ... -2.275 -1.7
    mf_iv_chi2            (run, probe, theta) float64 nan nan ... 763.6 708.5
    mf_iv_r

In [22]:
combined_ds.sel(run='hg_fflwp', probe='flat_flush')['ion_voltage_max'].values

KeyError: 'hg_fflwp'

In [22]:
fig, ax = plt.subplots(2)
combined_ds.sel(run=['fflwp', 'fwp_0'], probe='angled_recessed')['ion_a'].plot(x='theta', hue='run', marker='s', mfc='none', ax=ax[0])
combined_ds.sel(run=['fflwp', 'fwp_0'], probe='flat_flush')['str_iv_a'].plot(x='theta', hue='run', marker='s', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(2)
combined_ds.sel(run=['fflwp', 'fwp_2', 'fwp_0'], probe='flat_flush')['ion_a'].plot(x='theta', hue='run', marker='s', ax=ax[0])
combined_ds.sel(run=['fflwp', 'fwp_2', 'fwp_0'], probe='angled_recessed')['ion_a'].plot(x='theta', hue='run', marker='s', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(2)
combined_ds.sel(run=['hg_fflwp', 'hg'], probe='flat_flush')['ion_a'].plot(x='theta', hue='run', marker='s', ax=ax[0])
combined_ds.sel(run=['hg_fflwp', 'hg'], probe='angled_recessed')['ion_a'].plot(x='theta', hue='run', marker='s', ax=ax[1])

In [45]:
combined_ds.sel(run='fflwp', probe='angled_recessed', theta=slice(10,30))['current_i'].plot(x='voltage', hue='theta')

In [21]:
combined_ds.to_netcdf('se_combined.nc')

### Analysis will take place in other notebooks, likely analysis_3.ipynb